In [1]:
import pandas as pd

In [2]:
file_path = 'SentiWord_Dict.txt'

In [3]:
# 사전 데이터 불러오기
df = pd.read_csv('SentiWord_Dict.txt', delimiter='\t', names=['단어', '점수'], header=None)

In [4]:
# 한글 부분만 필요하기 때문에, 사전에서 한글필터링 - 정규표현식
df = df[df['단어'].str.match(r'^[가-힣]+$')]

In [22]:
df.head(10)

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0
50,가난뱅이,-2.0
51,가난살이,-2.0
52,가난살이하다,-2.0
53,가난설음,-2.0
54,가난에,-2.0


In [23]:
positive_words= ['상승','호재','수익','매수','강세',
                 '성장','기대감','안정성','배당','흥행',
                 '신규','투자','우량주','전망','실적',
                 '신뢰','회복','대박','상한가','수익률',
                 '투더문','떡상','상승세','신고가','역대급',
                 '황금주','대세','상승','급등','발전',
                 '전환점','인수합병','신제품','신기술','비전',
                 '선도','전망치','잭팟','우상향','강세장','화성']

In [27]:
positive_df = pd.DataFrame({'단어': positive_words, '점수': [1.0] * len(positive_words)})

In [31]:
positive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      41 non-null     object 
 1   점수      41 non-null     float64
dtypes: float64(1), object(1)
memory usage: 788.0+ bytes


In [24]:
negative_words = ['손절','폭락','물렸다','하한가','떡락',
                  '개미털기','작전주','버블','폭탄주','축소',
                  '물타기','매도','공매도','붕괴','실패','하락',
                  '피눈물','악재','손해','물로켓','부도','리콜',
                  '변동성','퇴출','파산','잠식','위기','침체',
                  '불황','나락','폭탄','이탈','손실','냉각',
                  '마이너스','폭망','미달'
                  ]

In [32]:
negative_df = pd.DataFrame({'단어': negative_words, '점수': [-1.0] * len(negative_words)})

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6384 entries, 44 to 14854
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6384 non-null   object 
 1   점수      6384 non-null   float64
dtypes: float64(1), object(1)
memory usage: 149.6+ KB


In [29]:
df = pd.concat([df, positive_df], ignore_index=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425 entries, 0 to 6424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6425 non-null   object 
 1   점수      6425 non-null   float64
dtypes: float64(1), object(1)
memory usage: 100.5+ KB


In [34]:
df = pd.concat([df, negative_df], ignore_index=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6462 entries, 0 to 6461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6462 non-null   object 
 1   점수      6462 non-null   float64
dtypes: float64(1), object(1)
memory usage: 101.1+ KB


In [36]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')

In [37]:
df.head(5)

,단어,점수
0,가까스로,0.0
1,가까이하다,1.0
2,가꾸러뜨리다,-1.0
3,가꾸러트리다,-1.0
4,가난,-2.0


In [39]:
df = df[df['단어'].str.len() > 1]

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6436 entries, 0 to 6461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6436 non-null   object 
 1   점수      6436 non-null   float64
dtypes: float64(1), object(1)
memory usage: 150.8+ KB


In [41]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')

In [6]:
community_contents = pd.read_csv('community_token_final.csv')

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\287724821.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  community_contents = pd.read_csv('community_token_final.csv')


In [15]:
# 데이터 타입 확인
print(community_contents.dtypes)

닉네임           object
날짜            object
내용            object
종목            object
내용_명사         object
내용_형용사        object
내용_부사         object
내용_동사         object
내용_명사_str     object
내용_형용사_str    object
내용_부사_str     object
내용_동사_str     object
dtype: object


In [7]:
community_contents = pd.read_csv('community_token_final.csv', low_memory=False)

In [17]:
# 데이터 타입 확인
print(community_contents.dtypes)

닉네임           object
날짜            object
내용            object
종목            object
내용_명사         object
내용_형용사        object
내용_부사         object
내용_동사         object
내용_명사_str     object
내용_형용사_str    object
내용_부사_str     object
내용_동사_str     object
dtype: object


In [8]:
community_contents.head(2)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다"
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있"


df.head()

In [22]:
df.head()

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0


In [23]:
def calculate_positive_score(contents):
  total_score = 0.0
  
  positive_dict = pd.read_csv('positive_score_dict.csv')
  positive_scores = []
  
  for nouns in contents['내용_명사']:
    total_score = 0.0

    for noun in nouns:  # nouns는 리스트
        noun_score = positive_dict.loc[positive_dict['단어'] == noun, '점수']
        if not noun_score.empty:
            total_score += noun_score.values[0]  # 점수 합산
    positive_scores.append(total_score)
  
  for adjectives in contents['내용_형용사']:
    for adjective in adjectives:  
        adj_score = positive_dict.loc[positive_dict['단어'] == adjective, '점수']
        if not adj_score.empty:
            total_score += adj_score.values[0]  # 점수 합산
    positive_scores.append(total_score)
  
  for adverbs in contents['내용_부사']:
    for adverb in adverbs:  
        adv_score = positive_dict.loc[positive_dict['단어'] == adverb, '점수']
        if not adv_score.empty:
            total_score += adv_score.values[0]  # 점수 합산
    positive_scores.append(total_score)
  
  for verbs in contents['내용_동사']:
    for verb in verbs:  
        verb_score = positive_dict.loc[positive_dict['단어'] == verb, '점수']
        if not verb_score.empty:
            total_score += verb_score.values[0]  # 점수 합산
    positive_scores.append(total_score)  
  
  
  # 긍정 점수에 따라 새로운 열 추가
  contents['긍정점수'] = [1 if score > 0 else (0 if score == 0 else -1) for score in positive_scores]
  
  return contents

In [42]:
def calculate_positive_score(contents):
    
  positive_dict = pd.read_csv('positive_score_dict.csv')
  
  def calculate_score(words):
        total_score = 0.0
        for word in words:
            word_score = positive_dict.loc[positive_dict['단어'] == word, '점수']
            if not word_score.empty:
                total_score += word_score.values[0]  # 점수 합산
                print(f"더해지는 단어: {word}, 점수: {word_score.values[0]}")  # 단어와 점수 출력
        return total_score

  positive_scores = []

  for index, row in contents.iterrows():
      total_score = 0.0
      total_score += calculate_score(row['내용_명사'])
      total_score += calculate_score(row['내용_형용사'])
      total_score += calculate_score(row['내용_부사'])
      total_score += calculate_score(row['내용_동사'])
      positive_scores.append(total_score)

  # 긍정 점수에 따라 새로운 열 추가
  contents['긍정점수'] = [1 if score > 0 else (0 if score == 0 else -1) for score in positive_scores]

  return contents
    

In [43]:
tmp = community_contents.head(30)

In [44]:
tmp.head(1)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다"


In [45]:
# 20분소요
result = calculate_positive_score(tmp)

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\2363172548.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents['긍정점수'] = [1 if score > 0 else (0 if score == 0 else -1) for score in positive_scores]


In [ ]:
result

In [6]:
result['긍정점수_명사'].value_counts()

NameError: name 'result' is not defined

In [23]:
# 사전에 없는 단어를 저장할 집합
missing_nouns = set()

In [24]:
existing_words = set(df['단어'])

In [25]:
# 모든 행을 돌면서 사전에 없는 단어 찾기
for nouns in community_contents['내용_명사']:
    for noun in nouns:  # 내부 리스트의 각 단어 순회
        clean_noun = noun.strip()  # 공백 제거
        if len(clean_noun) > 1 and clean_noun not in existing_words:  # 1글자 단어 제외
            missing_nouns.add(clean_noun)

In [26]:
missing_nouns

set()

In [51]:
# 특정 단어 확인
specific_word = '디자인'  # 확인하고자 하는 단어

if specific_word in existing_words:
    print(f"'{specific_word}'는 existing_words에 존재합니다.")
else:
    print(f"'{specific_word}'는 existing_words에 존재하지 않습니다.")

'디자인'는 existing_words에 존재하지 않습니다.


In [ ]:
existing_words = set(df['단어'])  # 중복 제거를 위해 set으로 변환

# 사전에 없는 단어를 저장할 집합
missing_nouns = set()

# 모든 행을 돌면서 사전에 없는 단어 찾기
for nouns in result['내용_명사']:
    for noun in nouns:
        if noun not in existing_words:
            missing_nouns.add(noun)

In [61]:
# 데이터프레임 불러오기
community_contents = pd.read_csv('community_token_final.csv')  # 데이터 파일 경로 입력

# 긍정 점수 사전 읽기
positive_dict = pd.read_csv('positive_score_dict.csv')

# 중복된 행 제거
positive_dict = positive_dict.drop_duplicates()

# 단어를 인덱스로 설정
positive_dict.set_index('단어', inplace=True)

# 긍정 점수 계산 함수
def calculate_positive_score(row):
    total_score = 0.0
    for word_list in [row['내용_명사'], row['내용_형용사'], row['내용_부사'], row['내용_동사']]:
        for word in word_list:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                total_score += positive_dict.loc[word, '점수']
    return total_score

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\3557543317.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  community_contents = pd.read_csv('community_token_final.csv')  # 데이터 파일 경로 입력


In [67]:
# 데이터프레임 불러오기
community_contents = pd.read_csv('community_token_final.csv')  # 데이터 파일 경로 입력

# 긍정 점수 사전 읽기
positive_dict = pd.read_csv('positive_score_dict.csv')

# 중복된 행 제거
positive_dict = positive_dict.drop_duplicates()

# 단어를 인덱스로 설정
positive_dict.set_index('단어', inplace=True)

# 긍정 점수 계산 함수
def calculate_positive_score(row):
    total_score = 0.0
    for word_list in row['내용_명사'] :
        for word in word_list:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                total_score += positive_dict.loc[word, '점수']
    return total_score

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\2403873552.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  community_contents = pd.read_csv('community_token_final.csv')  # 데이터 파일 경로 입력


In [94]:
tmp = community_contents.head(50)

In [76]:
tmp

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다"
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있"
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다"
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다"


In [77]:
positive_dict.head()

,점수
단어,
가까스로,0.0
가까이하다,1.0
가꾸러뜨리다,-1.0
가꾸러트리다,-1.0
가난,-2.0


In [83]:
tmp['내용_명사_str'][0]

'디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자인, 변화, 플러스, 외관, 크기, 차이, 외, 거의, 디자인, 프로세서, 전, 모델, 갤럭시, 용, 퀄컴, 스냅드래곤, 엘리트, 칩셋, 탑재, 기본, 용량, 증가, 전, 모델, 기본, 탑재, 울트라, 모델, 일부, 지역, 탑재, 가능성, 및, 최신, 운영체제, 및, 사용자, 인터페이스, 탑재, 통신, 및, 지원'

In [ ]:
tmp = 

In [87]:
row_scores = []


for idx, word_list in enumerate(tmp['내용_명사_str']):  # 인덱스와 값 가져오기
    row_score = 0
    print(f"\n행 {idx} 처리 중: {word_list}")
    if pd.isna(word_list):  # Null 값 처리
        print(f"행 {idx}는 결측값입니다. 건너뜁니다.")
        row_scores.append(row_score)  # 점수 0 저장
        continue
    
    words = word_list.split(', ')  # 쉼표와 공백으로 구분
    for word in words:
        word = word.strip()  # 공백 제거
        if word in positive_dict.index:
            score = positive_dict.loc[word, '점수']
            row_score += score  # 점수 합산
            print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
        # else:
        #     print(f"  단어: {word}는 사전에 없음")

    row_scores.append(row_score)  # 행별 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 모든 행 점수 출력
print("\n행별 총 긍정 점수:")
for idx, score in enumerate(row_scores):
    print(f"행 {idx}: {score}")



행 0 처리 중: 디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자인, 변화, 플러스, 외관, 크기, 차이, 외, 거의, 디자인, 프로세서, 전, 모델, 갤럭시, 용, 퀄컴, 스냅드래곤, 엘리트, 칩셋, 탑재, 기본, 용량, 증가, 전, 모델, 기본, 탑재, 울트라, 모델, 일부, 지역, 탑재, 가능성, 및, 최신, 운영체제, 및, 사용자, 인터페이스, 탑재, 통신, 및, 지원
  단어: 엘리트, 점수: 1.0, 누적 점수: 1.0
행 0의 총 점수: 1.0

행 1 처리 중: 대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시
  단어: 회복, 점수: 1.0, 누적 점수: 1.0
  단어: 매도, 점수: -1.0, 누적 점수: 0.0
행 1의 총 점수: 0.0

행 2 처리 중: 분할, 타점, 얘, 얘, 살바, 살, 단점
행 2의 총 점수: 0

행 3 처리 중: 존버하
행 3의 총 점수: 0

행 4 처리 중: 제, 오늘, 일, 퇴근
행 4의 총 점수: 0

행 5 처리 중: 하나, 과거, 지도, 사람, 투자, 사람, 명, 사람, 포기, 다그, 세상, 누구, 기대, 뭔가, 듯이, 한번, 번, 세상, 노력, 도저히, 안, 더, 기본, 인식, 깨, 말
  단어: 투자, 점수: 1.0, 누적 점수: 1.0
  단어: 포기, 점수: -1.0, 누적 점수: 0.0
  단어: 기대, 점수: 1.0, 누적 점수: 1.0
행 5의 총 점수: 1.0

행 6 처리 중: 정치, 관심, 이제, 공부, 윤석열, 대통령, 틴핵, 선진국, 삼상, 전자, 주식, 수
행 6의 총 점수: 0

행 7 처리 중: nan
행 7는 결측값입니다. 건너뜁니다.

행 8 처리 중: 요즘, 왜, 미세먼지, 중국, 중이, 친환경
행 8의 총 점수: 0

행 9 처리 중: 뭔가, 듯이, 한번, 번, 세상, 노력, 도저히, 안, 더, 기본, 인식, 깨, 말, 재
행 9의 총 점수: 0

행 10 처리 중: 거, 소프트, 더, 전망, 주식
  단어: 전망,

In [88]:
row_scores = []


for idx, word_list in enumerate(tmp['내용_형용사_str']):  # 인덱스와 값 가져오기
    row_score = 0
    print(f"\n행 {idx} 처리 중: {word_list}")
    if pd.isna(word_list):  # Null 값 처리
        print(f"행 {idx}는 결측값입니다. 건너뜁니다.")
        row_scores.append(row_score)  # 점수 0 저장
        continue
    
    words = word_list.split(', ')  # 쉼표와 공백으로 구분
    for word in words:
        word = word.strip()  # 공백 제거
        if word in positive_dict.index:
            score = positive_dict.loc[word, '점수']
            row_score += score  # 점수 합산
            print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
        # else:
        #     print(f"  단어: {word}는 사전에 없음")

    row_scores.append(row_score)  # 행별 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 모든 행 점수 출력
print("\n행별 총 긍정 점수:")
for idx, score in enumerate(row_scores):
    print(f"행 {idx}: {score}")


행 0 처리 중: 둥근, 있습니다, 동일한, 있음
행 0의 총 점수: 0

행 1 처리 중: nan
행 1는 결측값입니다. 건너뜁니다.

행 2 처리 중: 좋은, 느리고, 많다는
  단어: 좋은, 점수: 2.0, 누적 점수: 2.0
행 2의 총 점수: 2.0

행 3 처리 중: nan
행 3는 결측값입니다. 건너뜁니다.

행 4 처리 중: nan
행 4는 결측값입니다. 건너뜁니다.

행 5 처리 중: 없고, 안되는, 있는데, 좋겠어, 그런
행 5의 총 점수: 0

행 6 처리 중: 없는데, 있을까요
행 6의 총 점수: 0

행 7 처리 중: nan
행 7는 결측값입니다. 건너뜁니다.

행 8 처리 중: 없냐, 망하는, 아니면
  단어: 망하는, 점수: -2.0, 누적 점수: -2.0
행 8의 총 점수: -2.0

행 9 처리 중: 그런
행 9의 총 점수: 0

행 10 처리 중: 좋은
  단어: 좋은, 점수: 2.0, 누적 점수: 2.0
행 10의 총 점수: 2.0

행 11 처리 중: 그럴려면, 높은, 없다는, 엄청난, 필요하니까, 좋으라고, 자명하다, 같긴한데
  단어: 높은, 점수: 1.0, 누적 점수: 1.0
행 11의 총 점수: 1.0

행 12 처리 중: nan
행 12는 결측값입니다. 건너뜁니다.

행 13 처리 중: 없다는
행 13의 총 점수: 0

행 14 처리 중: nan
행 14는 결측값입니다. 건너뜁니다.

행 15 처리 중: 많아서, 힘들었는데, 없네
행 15의 총 점수: 0

행 16 처리 중: 아니고, 많아서, 같은데, 없는
행 16의 총 점수: 0

행 17 처리 중: 어떻게, 있을까요
행 17의 총 점수: 0

행 18 처리 중: 안되지만, 없네요, 많은
행 18의 총 점수: 0

행 19 처리 중: 비슷한거, 같은데
행 19의 총 점수: 0

행 20 처리 중: nan
행 20는 결측값입니다. 건너뜁니다.

행 21 처리 중: 좋은데
행 21의 총 점수: 0

행 22 처리 중: nan
행 22는 결측값입니다. 건너뜁니다.



In [89]:
row_scores = []


for idx, word_list in enumerate(tmp['내용_부사_str']):  # 인덱스와 값 가져오기
    row_score = 0
    print(f"\n행 {idx} 처리 중: {word_list}")
    if pd.isna(word_list):  # Null 값 처리
        print(f"행 {idx}는 결측값입니다. 건너뜁니다.")
        row_scores.append(row_score)  # 점수 0 저장
        continue
    
    words = word_list.split(', ')  # 쉼표와 공백으로 구분
    for word in words:
        word = word.strip()  # 공백 제거
        if word in positive_dict.index:
            score = positive_dict.loc[word, '점수']
            row_score += score  # 점수 합산
            print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
        # else:
        #     print(f"  단어: {word}는 사전에 없음")

    row_scores.append(row_score)  # 행별 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 모든 행 점수 출력
print("\n행별 총 긍정 점수:")
for idx, score in enumerate(row_scores):
    print(f"행 {idx}: {score}")


행 0 처리 중: nan
행 0는 결측값입니다. 건너뜁니다.

행 1 처리 중: nan
행 1는 결측값입니다. 건너뜁니다.

행 2 처리 중: 너무, 오래, 너무
행 2의 총 점수: 0

행 3 처리 중: nan
행 3는 결측값입니다. 건너뜁니다.

행 4 처리 중: 열심히
  단어: 열심히, 점수: 1.0, 누적 점수: 1.0
행 4의 총 점수: 1.0

행 5 처리 중: 이렇게, 그걸
행 5의 총 점수: 0

행 6 처리 중: nan
행 6는 결측값입니다. 건너뜁니다.

행 7 처리 중: nan
행 7는 결측값입니다. 건너뜁니다.

행 8 처리 중: nan
행 8는 결측값입니다. 건너뜁니다.

행 9 처리 중: 그걸
행 9의 총 점수: 0

행 10 처리 중: nan
행 10는 결측값입니다. 건너뜁니다.

행 11 처리 중: 한동안, 적어도
행 11의 총 점수: 0

행 12 처리 중: nan
행 12는 결측값입니다. 건너뜁니다.

행 13 처리 중: nan
행 13는 결측값입니다. 건너뜁니다.

행 14 처리 중: nan
행 14는 결측값입니다. 건너뜁니다.

행 15 처리 중: 다
행 15의 총 점수: 0

행 16 처리 중: nan
행 16는 결측값입니다. 건너뜁니다.

행 17 처리 중: nan
행 17는 결측값입니다. 건너뜁니다.

행 18 처리 중: 이렇게
행 18의 총 점수: 0

행 19 처리 중: nan
행 19는 결측값입니다. 건너뜁니다.

행 20 처리 중: nan
행 20는 결측값입니다. 건너뜁니다.

행 21 처리 중: nan
행 21는 결측값입니다. 건너뜁니다.

행 22 처리 중: nan
행 22는 결측값입니다. 건너뜁니다.

행 23 처리 중: nan
행 23는 결측값입니다. 건너뜁니다.

행 24 처리 중: 너무
행 24의 총 점수: 0

행 25 처리 중: nan
행 25는 결측값입니다. 건너뜁니다.

행 26 처리 중: nan
행 26는 결측값입니다. 건너뜁니다.

행 27 처리 중: nan
행 27는 결측값입니다. 건너

In [90]:
row_scores = []


for idx, word_list in enumerate(tmp['내용_동사_str']):  # 인덱스와 값 가져오기
    row_score = 0
    print(f"\n행 {idx} 처리 중: {word_list}")
    if pd.isna(word_list):  # Null 값 처리
        print(f"행 {idx}는 결측값입니다. 건너뜁니다.")
        row_scores.append(row_score)  # 점수 0 저장
        continue
    
    words = word_list.split(', ')  # 쉼표와 공백으로 구분
    for word in words:
        word = word.strip()  # 공백 제거
        if word in positive_dict.index:
            score = positive_dict.loc[word, '점수']
            row_score += score  # 점수 합산
            print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
        # else:
        #     print(f"  단어: {word}는 사전에 없음")

    row_scores.append(row_score)  # 행별 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 모든 행 점수 출력
print("\n행별 총 긍정 점수:")
for idx, score in enumerate(row_scores):
    print(f"행 {idx}: {score}")


행 0 처리 중: 되었으며, 와, 보입니다
행 0의 총 점수: 0

행 1 처리 중: 물려있는게, 올라도, 물려있
행 1의 총 점수: 0

행 2 처리 중: 매수하기에, 걸린다
행 2의 총 점수: 0

행 3 처리 중: 오르겠죠
행 3의 총 점수: 0

행 4 처리 중: 오를, 겁니다, 했습니다
행 4의 총 점수: 0

행 5 처리 중: 믿을거, 모르는, 믿을을주는, 할, 까야, 바뀐다고, 했음, 쪽팔려, 보여주고, 싶지, 않아, 보란, 바뀌고, 싶지, 않아, 한, 보여줘, 해도, 되, 달란
  단어: 모르는, 점수: -1.0, 누적 점수: -1.0
행 5의 총 점수: -1.0

행 6 처리 중: 하려고, 하는데, 되어야, 되고, 오를
행 6의 총 점수: 0

행 7 처리 중: nan
행 7는 결측값입니다. 건너뜁니다.

행 8 처리 중: 바꾼거냐
행 8의 총 점수: 0

행 9 처리 중: 보여주고, 싶지, 않아, 보란, 바뀌고, 싶지, 않아, 한, 보여줘, 해도, 되, 달란, 용아
행 9의 총 점수: 0

행 10 처리 중: nan
행 10는 결측값입니다. 건너뜁니다.

행 11 처리 중: 보면, 내려가야, 올라타고, 나온다, 뺏다, 올렸다하면서, 낮춰야, 된다, 털어내야, 낮춰진다, 쌓여있어서, 줄수가, 큰, 올릴려면, 주면, 털기, 서라도, 할게, 큰, 두고있는거, 대출받아, 버틴다, 하면, 떨어지든, 오르든, 냅, 두면, 올라있을
행 11의 총 점수: 0

행 12 처리 중: 하면, 되나요
행 12의 총 점수: 0

행 13 처리 중: 흔들려, 하시는, 되더라
행 13의 총 점수: 0

행 14 처리 중: 였었나
행 14의 총 점수: 0

행 15 처리 중: 해도, 넘, 숨도, 쉬기, 멈췄나
행 15의 총 점수: 0

행 16 처리 중: 될, 겁니다, 털어, 안줄, 겁니다, 지쳐, 떨어질, 가져가도, 빠지게, 하는거, 보면, 빠지면, 털기, 일거, 떨어질, 잃을수, 와있다고, 봐야겠네요
  단어: 지쳐, 점수: -1.0, 누적 점수: -1.0
행 16

In [95]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = []  # 총 점수 저장 리스트

for idx in range(len(tmp)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '내용_형용사_str', '내용_부사_str', '내용_동사_str']:
        word_list = tmp.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list):  # Null 값 처리
            print(f"행 {idx}, 열 {column}은 결측값입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            # else:
            #     print(f"  단어: {word}는 사전에 없음")

    row_scores.append(row_score)  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    row_scores = list(row_scores)  # row_scores가 pandas.Series 객체일 수 있으므로 리스트로 변환

    tmp['긍정점수'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]

except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(tmp)}, row_scores 길이: {len(row_scores)}")


# 최종 점수 출력
print("\n행별 긍정점수:")
# print(row_scores)
print(tmp[['긍정점수']])


행 0, 열 내용_명사_str 처리 중: 디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자인, 변화, 플러스, 외관, 크기, 차이, 외, 거의, 디자인, 프로세서, 전, 모델, 갤럭시, 용, 퀄컴, 스냅드래곤, 엘리트, 칩셋, 탑재, 기본, 용량, 증가, 전, 모델, 기본, 탑재, 울트라, 모델, 일부, 지역, 탑재, 가능성, 및, 최신, 운영체제, 및, 사용자, 인터페이스, 탑재, 통신, 및, 지원
  단어: 엘리트, 점수: 1.0, 누적 점수: 1.0

행 0, 열 내용_형용사_str 처리 중: 둥근, 있습니다, 동일한, 있음

행 0, 열 내용_부사_str 처리 중: nan
행 0, 열 내용_부사_str은 결측값입니다. 건너뜁니다.

행 0, 열 내용_동사_str 처리 중: 되었으며, 와, 보입니다
행 0의 총 점수: 1.0

행 1, 열 내용_명사_str 처리 중: 대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시
  단어: 회복, 점수: 1.0, 누적 점수: 1.0
  단어: 매도, 점수: -1.0, 누적 점수: 0.0

행 1, 열 내용_형용사_str 처리 중: nan
행 1, 열 내용_형용사_str은 결측값입니다. 건너뜁니다.

행 1, 열 내용_부사_str 처리 중: nan
행 1, 열 내용_부사_str은 결측값입니다. 건너뜁니다.

행 1, 열 내용_동사_str 처리 중: 물려있는게, 올라도, 물려있
행 1의 총 점수: 0.0

행 2, 열 내용_명사_str 처리 중: 분할, 타점, 얘, 얘, 살바, 살, 단점

행 2, 열 내용_형용사_str 처리 중: 좋은, 느리고, 많다는
  단어: 좋은, 점수: 2.0, 누적 점수: 2.0

행 2, 열 내용_부사_str 처리 중: 너무, 오래, 너무

행 2, 열 내용_동사_str 처리 중: 매수하기에, 걸린다
행 2의 총 점수: 2.0

행 3, 열 내용_명사_str 처리 중: 존버하

행 3, 열 내용_형용사_str 처리 중: nan
행 3, 열 내

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\2989539696.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['긍정점수'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]


In [97]:
tmp.head(10)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다",1
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",0
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다",1
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠,0
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다",1
5,추팝츄스,2025-01-11T17:58:08+09:00,믿을거 하나없고 과거가 어떤지도 모르는 사람한테 투자한 사람이야 이렇게 믿을을주는 ...,5930,"['하나', '과거', '지도', '사람', '투자', '사람', '명', '사람'...","['없고', '안되는', '있는데', '좋겠어', '그런']","['이렇게', '그걸']","['믿을거', '모르는', '믿을을주는', '할', '까야', '바뀐다고', '했음...","하나, 과거, 지도, 사람, 투자, 사람, 명, 사람, 포기, 다그, 세상, 누구,...","없고, 안되는, 있는데, 좋겠어, 그런","이렇게, 그걸","믿을거, 모르는, 믿을을주는, 할, 까야, 바뀐다고, 했음, 쪽팔려, 보여주고, 싶...",0
6,복권1등지방,2025-01-11T17:43:07+09:00,정치에 관심이 없는데 이제부터 공부하려고 하는데 윤석열 대통령님이 틴핵되어야 선진국...,5930,"['정치', '관심', '이제', '공부', '윤석열', '대통령', '틴핵', '...","['없는데', '있을까요']",[],"['하려고', '하는데', '되어야', '되고', '오를']","정치, 관심, 이제, 공부, 윤석열, 대통령, 틴핵, 선진국, 삼상, 전자, 주식, 수","없는데, 있을까요",NaN,"하려고, 하는데, 되어야, 되고, 오를",0
7,삼전35만원,2025-01-11T17:26:18+09:00,https://youtu.be/f-8vau3NfMY?si=-dLlQD9Ph5OU1Wes,5930,[],[],[],[],NaN,NaN,NaN,NaN,0
8,분할매수매도장인,2025-01-11T16:46:27+09:00,요즘 왜 미세먼지가 없냐?? 중국이 망하는 중이냐? 아니면 친환경으로 바꾼거냐??,5930,"['요즘', '왜', '미세먼지', '중국', '중이', '친환경']","['없냐', '망하는', '아니면']",[],['바꾼거냐'],"요즘, 왜, 미세먼지, 중국, 중이, 친환경","없냐, 망하는, 아니면",NaN,바꾼거냐,-1
9,추팝츄스,2025-01-11T16:35:12+09:00,뭔가 보여주고 싶지 않아? 보란듯이 한번 바뀌고 싶지 않아? 그걸 한 번 보여줘!\...,5930,"['뭔가', '듯이', '한번', '번', '세상', '노력', '도저히', '안'...",['그런'],['그걸'],"['보여주고', '싶지', '않아', '보란', '바뀌고', '싶지', '않아', ...","뭔가, 듯이, 한번, 번, 세상, 노력, 도저히, 안, 더, 기본, 인식, 깨, 말, 재",그런,그걸,"보여주고, 싶지, 않아, 보란, 바뀌고, 싶지, 않아, 한, 보여줘, 해도, 되, ...",0


In [69]:
tmp['긍정점수'] = tmp.apply(calculate_positive_score, axis=1)

C:\Users\hydrl\AppData\Local\Temp\ipykernel_20264\2175701572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['긍정점수'] = tmp.apply(calculate_positive_score, axis=1)


In [70]:
tmp

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다",0.0
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",0.0
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다",0.0
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠,0.0
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다",0.0
5,추팝츄스,2025-01-11T17:58:08+09:00,믿을거 하나없고 과거가 어떤지도 모르는 사람한테 투자한 사람이야 이렇게 믿을을주는 ...,5930,"['하나', '과거', '지도', '사람', '투자', '사람', '명', '사람'...","['없고', '안되는', '있는데', '좋겠어', '그런']","['이렇게', '그걸']","['믿을거', '모르는', '믿을을주는', '할', '까야', '바뀐다고', '했음...","하나, 과거, 지도, 사람, 투자, 사람, 명, 사람, 포기, 다그, 세상, 누구,...","없고, 안되는, 있는데, 좋겠어, 그런","이렇게, 그걸","믿을거, 모르는, 믿을을주는, 할, 까야, 바뀐다고, 했음, 쪽팔려, 보여주고, 싶...",0.0
6,복권1등지방,2025-01-11T17:43:07+09:00,정치에 관심이 없는데 이제부터 공부하려고 하는데 윤석열 대통령님이 틴핵되어야 선진국...,5930,"['정치', '관심', '이제', '공부', '윤석열', '대통령', '틴핵', '...","['없는데', '있을까요']",[],"['하려고', '하는데', '되어야', '되고', '오를']","정치, 관심, 이제, 공부, 윤석열, 대통령, 틴핵, 선진국, 삼상, 전자, 주식, 수","없는데, 있을까요",NaN,"하려고, 하는데, 되어야, 되고, 오를",0.0
7,삼전35만원,2025-01-11T17:26:18+09:00,https://youtu.be/f-8vau3NfMY?si=-dLlQD9Ph5OU1Wes,5930,[],[],[],[],NaN,NaN,NaN,NaN,0.0
8,분할매수매도장인,2025-01-11T16:46:27+09:00,요즘 왜 미세먼지가 없냐?? 중국이 망하는 중이냐? 아니면 친환경으로 바꾼거냐??,5930,"['요즘', '왜', '미세먼지', '중국', '중이', '친환경']","['없냐', '망하는', '아니면']",[],['바꾼거냐'],"요즘, 왜, 미세먼지, 중국, 중이, 친환경","없냐, 망하는, 아니면",NaN,바꾼거냐,0.0
9,추팝츄스,2025-01-11T16:35:12+09:00,뭔가 보여주고 싶지 않아? 보란듯이 한번 바뀌고 싶지 않아? 그걸 한 번 보여줘!\...,5930,"['뭔가', '듯이', '한번', '번', '세상', '노력', '도저히', '안'...",['그런'],['그걸'],"['보여주고', '싶지', '않아', '보란', '바뀌고', '싶지', '않아', ...","뭔가, 듯이, 한번, 번, 세상, 노력, 도저히, 안, 더, 기본, 인식, 깨, 말, 재",그런,그걸,"보여주고, 싶지, 않아, 보란, 바뀌고, 싶지, 않아, 한, 보여줘, 해도, 되, ...",0.0


In [57]:
# 중복된 행 제거
positive_dict = positive_dict.drop_duplicates()